In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 16,kernel_size=7, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32,kernel_size=3, bias=False)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 10, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(10)
        self.gap = nn.AvgPool2d(10)

    def forward(self, x):
        # C1:n_i = 28, n_o = 22, s = 1, j_i = 1, j_o = 1, r_i = 1, r_o = 7
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        # C2:n_i = 22, n_o = 20, s = 1, j_i = 1, j_o = 1, r_i = 7, r_o = 9
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        # M1:n_i = 20, n_o = 10, s = 2, j_i = 1, j_o = 2, r_i = 9, r_o = 10
        x = F.relu(F.max_pool2d(x,2))
        # C4:n_i = 10, n_o = 20, s = 1, j_i = 2, j_o = 2, r_i = 10, r_o = 12
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = self.gap(x)
        x = x.view(-1,10)
        return F.log_softmax(x,dim=1)

In [15]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 22, 22]             784
            Conv2d-2           [-1, 32, 20, 20]           4,608
            Conv2d-3           [-1, 10, 10, 10]             320
         AvgPool2d-4             [-1, 10, 1, 1]               0
Total params: 5,712
Trainable params: 5,712
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.16
Params size (MB): 0.02
Estimated Total Size (MB): 0.19
----------------------------------------------------------------


In [16]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [18]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]